#### Importing and Installing necessary dependencies

In [ ]:
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from sklearn.decomposition import PCA
import random
import time

#### Downloading the dataset from the website but I have already downloaded dataset. So have to only synchronised the dataset folder path name

In [ ]:
!echo "Downloading 101_Object_Categories from caltech-101 for imagesearch"
!curl -L -o 101_ObjectCategories.tar.gz --progress-bar http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
!tar -xzf 101_ObjectCategories.tar.gz
!rm 101_ObjectCategories.tar.gz
!ls

#### Calling the inbuilt trained model of convolution neural netwrok from keras.

In [ ]:
model = keras.applications.VGG16(weights='imagenet',include_top=True)

In [ ]:
# Cheoking the featuires of traiend model
model.summary()

#### Checking the our image directory path.

In [ ]:
!ls /content/Rimage.jpeg

#### Defining the method for training the model.

In [ ]:
def load_image(path):
  img = image.load_img(path, target_size=model.input_shape[1:3])
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  return img,x


#### Loading the our images and calling the above defiend method

In [ ]:
img,x = load_image("/content/Rimage.jpeg")
print("shape of x: ",x.shape)
print("data type: ",x.dtype)
plt.imshow(img)

#### Predicting the image value

In [ ]:
predictions = model.predict(x)
# print(predictions)
for _, pred,prob in decode_predictions(predictions)[0]:
  print("predicted %s with probability %0.3f" %(pred,prob))

#### Extracting the features 

In [ ]:
feat_extractor = Model(inputs=model.input,outputs=model.get_layer("fc2").output)
feat_extractor.summary()

#### Plotting the image graph for features analysis

In [ ]:
img,x = load_image("/content/Rimage.jpeg")
feat = feat_extractor.predict(x)
plt.figure(figsize=(16,4))
plt.plot(feat[0])

#### Loading and checkig the number of image iterations

In [ ]:
images_path = '/content/Rimage.jpeg'
image_extensions=['.jpg','.png','.jpeg']
#keep the number less than 30 if running on cpu for faster results
max_num_images = 1000
tick = time.time()
# dp has the root directory,dn has the sub directories and filenames has the file name
images = '/content/Rimage.jpeg'
#images = [os.path.join(dp,f) for dp,dn,filenames in os.walk('/content/Rimage.jpeg') for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
if max_num_images<len(images):
#get random images from the image paths
  images = [images[i] for i in sorted(random.sample(range(len(images)),max_num_images))] #comment this and uncomment the below line if running on cpu to get better matches
#images = images[0:max_num_images]
print("keeping %d images to analyze" %len(images))

#### Image analysing process

In [ ]:
tic = time.time()
features = []
for i, image_path in enumerate(images):
 if i%500 == 0:
    toc = time.time()
    elap = toc-tic;
    print("analyzing image %d / %d. Time taken : %4.4f seconds"%(i,len(images),elap))
    tic= time.time()
 img,x = load_image('/content/Rimage.jpeg')
 feat = feat_extractor.predict(x)[0]
 features.append(feat)
print('finished extracting features for %d images' % len(images))

#### Principal component analysis

In [ ]:
features = np.array(features)
pca = PCA(n_components=20)
pca.fit(features)

In [ ]:
pca_features = pca.transform(features)

#### Plotting the PCA of image

In [ ]:
plt.plot(pca_features[2])

#### Checking the clock time

In [ ]:
tock = time.time()
print(tock-tick)

In [ ]:
def get_closest_images(query_image_idx, num_results=5):
    distances = [ distance.cosine(pca_features[query_image_idx], feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:num_results+1]
    return idx_closest

#### Defining the closet image and model buling and matching

In [ ]:
def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

#### Plotting the reverse image tarined model of our image dataset

In [ ]:
import random
query_image_idx = int(len(images)*random.random())
idx_closest = get_closest_images(query_image_idx)
#query_image = get_concatenated_images([query_image_idx],20)
#results_image = get_concatenated_images(idx_closest,20)

plt.figure(figsize=(5,5))
plt.imshow(img)
plt.title("query image(%d)" %query_image_idx)

plt.figure(figsize=(16,22))
plt.imshow(img)
plt.title("Result images")